In [ ]:
import cosmoglobe

In [2]:
cosmoglobe

<module 'cosmoglobe' from '/opt/anaconda3/envs/cosmoglobe_test/lib/python3.8/site-packages/cosmoglobe/__init__.py'>

In [7]:
from cosmoglobe.sky import model_from_chain

ModuleNotFoundError: No module named 'cosmoglobe.chain'

In [ ]:
import astropy.units as u

In [4]:
import pathlib

In [5]:
chain_dir = pathlib.Path("/Users/metinsan/Documents/doktor/Cosmoglobe_test_data/")
chain = chain_dir / "chain_test.h5"

In [6]:
model = model_from_chain(chain, nside=256)


NameError: name 'model_from_chain' is not defined